<a href="https://colab.research.google.com/github/elemi10/7506-TP-Org-de-datos/blob/master/XGBoost_practica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **XGBoost**

In [1]:
# importo las librerias
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn import tree

In [2]:
# cargo el set de datos de pruebas "Boston"
from sklearn.datasets import load_boston
boston = load_boston()
print( "Type of boston dataset:", type(boston))
print(boston)

Type of boston dataset: <class 'sklearn.utils.Bunch'>
{'data': array([[6.3200e-03, 1.8000e+01, 2.3100e+00, ..., 1.5300e+01, 3.9690e+02,
        4.9800e+00],
       [2.7310e-02, 0.0000e+00, 7.0700e+00, ..., 1.7800e+01, 3.9690e+02,
        9.1400e+00],
       [2.7290e-02, 0.0000e+00, 7.0700e+00, ..., 1.7800e+01, 3.9283e+02,
        4.0300e+00],
       ...,
       [6.0760e-02, 0.0000e+00, 1.1930e+01, ..., 2.1000e+01, 3.9690e+02,
        5.6400e+00],
       [1.0959e-01, 0.0000e+00, 1.1930e+01, ..., 2.1000e+01, 3.9345e+02,
        6.4800e+00],
       [4.7410e-02, 0.0000e+00, 1.1930e+01, ..., 2.1000e+01, 3.9690e+02,
        7.8800e+00]]), 'target': array([24. , 21.6, 34.7, 33.4, 36.2, 28.7, 22.9, 27.1, 16.5, 18.9, 15. ,
       18.9, 21.7, 20.4, 18.2, 19.9, 23.1, 17.5, 20.2, 18.2, 13.6, 19.6,
       15.2, 14.5, 15.6, 13.9, 16.6, 14.8, 18.4, 21. , 12.7, 14.5, 13.2,
       13.1, 13.5, 18.9, 20. , 21. , 24.7, 30.8, 34.9, 26.6, 25.3, 24.7,
       21.2, 19.3, 20. , 16.6, 14.4, 19.4, 19.7, 20.5, 25

## convertimos el set de datos en un dataframe

In [6]:
import pandas as pd

data = pd.DataFrame(boston.data)
#convierto las columnas en lo features
data.columns=boston.feature_names

## agregamos el target al dataframe

In [7]:
data['PRICE']=boston.target

## separamos la variable a *predecir*

In [8]:
# separamos las variables, dejamos en X los atributos y en y la variable a predecir (price)
X,y=data.iloc[:,:-1],data.iloc[:,-1]

## creamos el set de entrenamiento y de test

In [9]:
# importamos la funcion train_test_split que permite separar un set que tengamos (en este caso en X donde tenemos los datos)
# separamos X en X_train y en X_test; mientras que a y lo separamos en y_train e y_test
from sklearn.model_selection import train_test_split

In [10]:
# en y_train e y_test van a estar los valores de TARGET para predecir
X_train, X_test, y_train, y_test =\
  train_test_split(X,y,test_size=0.2, random_state=123)
# seteamos random_state para que cada vez que lo corramos tengamos la misma division (?)

## splicamos **XGBoost**

## Hiper-parámetros


*   **learning_rate**: tasa de aprendizaje (valores chicos, porque valores a 1 nos pueden generar overfitting)
*   **max_depth**: maxima profundidad de cada arbol (manejamos la maxima posibilidad de cada arbol al igual que RandomForest), aunque va a ser mas acotado en XGBoost
*   **subsample**: % de muestras usadas para cada arbole (valor muy bajo, posible underfitting). Si tenemos un valor muy bajo podemos tener posbilidad de underfitting. Si usamos pocos ejemplos para cada arbol no va a poder entrenarse correctamente)
*   **colsample_bytree**: % de features usadas para cada arbol (valores muy altos, posible overfitting). Vamos a determinar que % de features totales va a utilizarse para construir cada unos de los arboles. Un valor muy alto puede arrojar un  error de overfitting. Tratar de evitar que cada arbol no vea todos los features
*   **n_estimators**: cant de arboles a construir
*   **objective**: funcion de error a utilizar (algunas: reg:squarederror para regresion, reg:logistic o binary:logistic para clasificacion). Para el proceso de aprendizaje.

###Parametros de regularizacion:

*   **gamma**: umbral para hacer split basado en la reduccion de error de hacer el nuevo split. 
*   **alpha**: regularizacion para los pesos de las hojas. Un valor mas alto genera una mayor regularizacion.
*   **lambda**: similar alpha pero para la sintonia fina.






## instanciamos el regresor de XGBoost

In [14]:
xg_reg=xgb.XGBRegressor(objective='reg:squarederror',
                        colsample_bytree=0.3, learning_rate=0.1,
                        max_depth=5, alpha=10, n_estimators =10
                        )

## entrenamos

In [15]:
xg_reg.fit(X_train,y_train)

XGBRegressor(alpha=10, base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.3, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=5, min_child_weight=1, missing=None, n_estimators=10,
             n_jobs=1, nthread=None, objective='reg:squarederror',
             random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             seed=None, silent=None, subsample=1, verbosity=1)

## predecimos

In [16]:
preds=xg_reg.predict(X_test)

## calculamos el error en la predicciones

In [17]:
rmse= np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

RMSE: 10.449300


## entrenando y prediciendo con 100 arboles